In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False
    
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor    

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)

In [3]:
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

In [4]:
from torchvision.datasets import MNIST

download_root = './data/MNIST_DATASET/'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=False)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=False)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST_DATASET/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST_DATASET/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST_DATASET/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST_DATASET/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST_DATASET/MNIST/raw



In [5]:
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [6]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [14]:
inputs, labels = next(iter(train_loader))

In [15]:
labels.size()

torch.Size([64])

In [16]:
inputs.size()

torch.Size([64, 1, 28, 28])

In [52]:
inputs =inputs.view(-1,28,28)

In [53]:
seq = inputs[:,1,:]

In [54]:
seq.size()

torch.Size([64, 28])

In [23]:
class LSTMCell(nn.Module):
    def __init__(self,input_size,hidden_size,bias=True):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        nn.Linear()
        self.x2h = nn.Linear(input_size, 4 *hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std,std)
    
    def forward(self,x,hidden): # (64,28)의 sequence 하나를 입력받는다
        hx, cx = hidden
        x = x.view(-1,x.size(1)) # (64,28)로 이쁘게 핀다

        gates = self.x2h(x) + self.h2h(hx) #(64,4 * 128)
        gates = gates.squeeze()
        ingate, forgetgate,cellgate,outgate = gates.chunk(4,1) #(64,28)

        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.sigmoid(cellgate)
        outgate = F.sigmoid(outgate)

        cy = torch.mul(cx, forgetgate) + torch.mul(ingate,cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        return (hy,cy)

MNIST는 손으로 쓴 숫자 이미지 데이터입니다. 하나의 이미지는 가로 28개, 세로 28개, 총 784개의 값으로 이루어져 있습니다.

Many-to-One model는 여러 시퀀스를 넣었을 때 나오는 최종 결과물만을 이용하는 모델입니다. 이를 이용하여 784개의 input으로 1개의 output값(A) 을 도출합니다. 이 A를 하나의 층에 통과시켜 10개의 숫자 label중 하나를 할당합니다.

784개의 입력값을 사이즈가 28인 벡터가 28번 이어지는 시퀀스(time step)로 보고, input의 크기를 28, 시퀀스 길이를 28로 각각 설정합니다. 28개의 input은 C라고 표현되어 있는 LSTM 셀로 순차적으로 들어가게 됩니다.

output의 크기는 셀의 크기와 같으며, 64로 설정하였습니다. 셀크기가 너무 작으면 많은 정보를 담지 못하기 때문에 적당히 큰 값으로 설정합니다. 전체 output은 64개의 값을 가지고 있는 벡터 28개의 집합이 되고, 마지막 벡터만 사용합니다.

1층의 fully connected layer를 이용하여 64차원 벡터를 10차원으로 줄이고 softmax를 이용하여 0부터 9까지 중 하나의 값을 예측합니다.

https://pozalabs.github.io/lstm/

In [25]:

class LSTMModel(nn.Module):
    def __init__(self,input_dim,hidden_dim,layer_dim,output_dim, bias=True):
        super().__init__()
        self.hidden_dim = hidden_dim

        self.layer_dim = layer_dim
        self.lstm = LSTMCell(input_dim,hidden_dim,layer_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self,x): # x: torch.Size([64, 28, 28]) (batch,seq_dim,input_dim)
        # 
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()) # (1,64,128)
        c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()) # (1,64,128)

        outs = []
        cn = c0[0,:,:]
        hn = h0[0,:,:]

        for seq in range(x.size(1)): # range(28)
            hn,cn = self.lstm(x[:,seq,:],(hn,cn)) # (64,1,28)로 sequence를 하나씩 전달
            outs.append(hn)
        
        out = outs[-1].squeeze()
        out = self.fc(out)
        return out

In [26]:
input_dim = 28
hidden_dim = 128
layer_dim = 1  
output_dim = 10
 
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available():
    model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [27]:
seq_dim = 28
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images = Variable(images.view(-1,seq_dim,input_dim).cuda()) #torch.Size([64, 1, 28, 28]) -> torch.Size([64, 28, 28])으로 변경
        labels = Variable(labels.cuda())

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.cuda()

        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        iter +=1

        if iter % 500 == 0:
            correct = 0
            total = 0

            for images, labels in valid_loader:
                images = Variable(images.view(-1,seq_dim,input_dim).cuda())
                outputs = model(images)
                _,predicted = torch.max(outputs.data,1)

                total += labels.size(0)
                correct += (predicted.cpu() == labels.cpu()).sum()
            
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

/home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration: 500. Loss: 2.318148374557495. Accuracy: 9.579999923706055
Iteration: 1000. Loss: 2.313196897506714. Accuracy: 10.279999732971191
Iteration: 1500. Loss: 2.301957130432129. Accuracy: 10.09000015258789
Iteration: 2000. Loss: 2.3158926963806152. Accuracy: 14.430000305175781
Iteration: 2500. Loss: 2.2820587158203125. Accuracy: 15.970000267028809
Iteration: 3000. Loss: 2.2391583919525146. Accuracy: 20.40999984741211
Iteration: 3500. Loss: 2.081110954284668. Accuracy: 23.860000610351562
Iteration: 4000. Loss: 2.08986234664917. Accuracy: 32.27000045776367
Iteration: 4500. Loss: 1.2805507183074951. Accuracy: 51.400001525878906
Iteration: 5000. Loss: 1.153741717338562. Accuracy: 65.5199966430664
Iteration: 5500. Loss: 0.8354787230491638. Accuracy: 71.55999755859375
Iteration: 6000. Loss: 0.92106032371521. Accuracy: 78.66000366210938
Iteration: 6500. Loss: 0.4569924771785736. Accuracy: 84.0
Iteration: 7000. Loss: 0.5676109790802002. Accuracy: 85.08000183105469
Iteration: 7500. Loss: 0.

In [37]:
def evaluate(model, val_iter):
    corrects, total, total_loss = 0,0,0
    model.eval()

    for images, labels in val_iter:
        images = Variable(images.view(-1,seq_dim,input_dim).cuda())
        
        labels = Variable(labels.cuda())
        
        logit = model(images).to(device)
        loss = F.cross_entropy(logit,labels,reduction='sum')
        
        _, pred = torch.max(logit.data, 1)
        total += labels.size(0)
        total_loss += loss.item()
        
        corrects += (pred == labels).sum()
    
    avg_loss = total_loss / len(val_iter)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy


In [38]:
test_loss, test_acc = evaluate(model, test_loader)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

Test Loss: 16.11 | Test Accuracy:  0.92
